這份程式碼是利用我的facebook的個人資訊(html)，並使用bs4去抓取資料。主題是我和朋友"戳一下"的紀錄，看我和哪個朋友互戳最多次

In [ ]:
!pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime

In [ ]:
with open('/content/pokes.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# 解析 HTML
soup = BeautifulSoup(html_content, 'html.parser')

# 提取所有事件
events = soup.find_all('div', class_="_a6-g")

# 存儲提取的數據
data = []
event_data = []

# 遍歷所有事件，提取名稱和時間
for event in events:
    name_div = event.find('div', class_="_2ph_ _a6-h _a6-i")
    time_div = event.find('div', class_="_a72d")

    if name_div and time_div:
        event_info = {
            'action': name_div.text,
            'timestamp': time_div.text
        }
        event_data.append(event_info)



In [ ]:
pattern = re.compile(r'(.+?)連?續?戳了(.+?)(?:\s\d+\s*次)?$')

for event in events:
    action_div = event.find('div', class_="_2ph_ _a6-h _a6-i")
    time_div = event.find('div', class_="_a72d")

    if action_div and time_div:
        action_text = action_div.text
        timestamp = time_div.text

        # 使用正則表達式提取戳人者和被戳者
        match = pattern.match(action_text)
        if match:
            stoker = match.group(1).strip()  # 戳人者
            stokee = match.group(2).strip()   # 被戳者

            # 將資料添加到列表中
            data.append({
                '戳人者': stoker,
                '被戳者': stokee,
                '時間': timestamp
            })

# 創建 DataFrame
df = pd.DataFrame(data)

# 打印結果，顯示戳人者和被戳者
print(df.head(50))

              戳人者            被戳者                         時間
0   Shih Yen Chen            郭大呈  10 月 07, 2021 11:29:21 上午
1             郭大呈  Shih Yen Chen   10 月 07, 2021 2:57:31 下午
2   Shih Yen Chen            郭大呈   10 月 13, 2021 8:02:05 上午
3             郭大呈  Shih Yen Chen  10 月 13, 2021 10:33:12 下午
4   Shih Yen Chen            郭大呈   10 月 14, 2021 8:09:18 上午
5             郭大呈  Shih Yen Chen   10 月 14, 2021 9:53:35 下午
6   Shih Yen Chen            郭大呈   10 月 15, 2021 4:35:03 下午
7             郭大呈  Shih Yen Chen  10 月 16, 2021 12:35:07 上午
8   Shih Yen Chen            郭大呈  10 月 16, 2021 12:42:53 上午
9             郭大呈  Shih Yen Chen   10 月 16, 2021 3:05:53 上午
10            郭大呈            林樂全   10 月 16, 2021 3:06:26 上午
11            郭大呈       Deca Nan   10 月 16, 2021 3:06:46 上午
12            郭大呈            邱昱誠   10 月 16, 2021 3:06:55 上午
13            郭大呈            楊祐承   10 月 16, 2021 3:07:21 上午
14            郭大呈            郭秉豐   10 月 16, 2021 3:07:49 上午
15            郭大呈            郭秉豐   10 月 

In [ ]:
def parse_date(date_str):
    # 將中文月份替換為數字月份
    month_map = {'1 月': '01', '2 月': '02', '3 月': '03', '4 月': '04', '5 月': '05', '6 月': '06',
                 '7 月': '07', '8 月': '08', '9 月': '09', '10 月': '10', '11 月': '11', '12 月': '12'}

    for k, v in month_map.items():
        date_str = date_str.replace(k, v)

    # 解析日期，根據格式 '%m %d, %Y %I:%M:%S %p' 進行解析
    return datetime.strptime(date_str, '%m %d, %Y %I:%M:%S %p')

# 應用解析函數到資料框中的'時間'列
df['時間'] = df['時間'].apply(parse_date)

print(df)

TypeError: strptime() argument 1 must be str, not NaTType

In [ ]:
filtered_df = df[df['被戳者'] != '郭大呈']
pd.set_option('display.max_rows', None)
# 使用 value_counts() 統計每個「被戳者」出現的次數
stokee_counts = filtered_df['被戳者'].value_counts()

# 打印結果
print(stokee_counts)

被戳者
陳妮妏              429
范淑敏              253
趙家佑              160
林樂全              129
江睿哲              123
許豐有              122
李采寧              102
王沁宇               97
Yumin Kao         96
楊祐承               85
黃允頎               82
呂佑震               79
林沁儒               65
楊恩綺               58
卜丹晴               45
陳聖繹               41
王辰宇               33
Wei Ting          30
Andy Lai          28
邱昱誠               27
PeiHong Sheng     24
齊婉廷               24
陳祥宇               21
李少妍               20
郭于慈               20
黃湘怡               20
魏子程               18
陳義豐               18
Sean Huang        15
李筱貞               15
Laurance Lin      15
李祐綦               15
陳崇軒               14
Shih Yen Chen     14
張淳麒               12
何東俊               10
陳宣瑜               10
許珽翰                9
曹靖倫                8
張于訓                8
林姿慧                8
李昱緯                8
許唐榮                7
Deca Nan           7
陳書安                6
張語哲                6
顏誠穎                6
Jerome Ch